Includes:
- User provides baseline yield. Therefore, output is not yield value, but a category indicating the likelihood of being above, below or betwen normal conditions
- Main chart is an stacked barplot showing the categories just described

In [1]:
import sys
# sys.path.remove('/home/dquintero/venvs/dssat/lib/python3.10/site-packages')
sys.path.append("/home/dquintero/venvs/serviceDSSAT/lib/python3.10/site-packages")
sys.path.append('/home/dquintero/spatialDSSAT')
sys.path.append('..')

In [2]:
from database import connect
from dssat import run_spatial_dssat
from data.transform import parse_overview
from datetime import datetime

In [3]:
import pandas as pd
import numpy as np

In [4]:
dbname = "dssatserv"

In [5]:
df, overview = run_spatial_dssat(
    dbname=dbname, 
    schema="kenya", 
    admin1="Nakuru",
    plantingdate=datetime(2022, 4, 1),
    cultivar="990002",
    nitrogen=[(5, 20), (30, 10), (50, 10)],
    overview=True
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 23.74it/s]


In [6]:
import ipywidgets as widgets

In [7]:
con = connect(dbname)

In [8]:
cur = con.cursor()
query = "SELECT admin1 FROM kenya.admin"
cur.execute(query)
admin1_list = [i[0] for i in cur.fetchall()]
cur.close()

In [9]:
cultivar_types = {'Short': "990003", 'Medium': "990002", 'Long': "990001"}

In [10]:
def run(pars):
    if "nitrogen_dap" in pars:
        nitro = list(zip(pars["nitrogen_dap"], pars["nitrogen_rate"]))
    else:
        nitro = [(0, pars["nitrogen"]),]
    df, overview = run_spatial_dssat(
            dbname=dbname, 
            schema="kenya", 
            admin1=pars["admin1"],
            plantingdate=datetime(pars["plantingdate"].year, pars["plantingdate"].month, pars["plantingdate"].day),
            cultivar=cultivar_types[pars["cultivar"]],
            nitrogen=nitro,
            overview=True,
            all_random=False
        )
    return df, overview

In [69]:
CURRENT_BASELINE_YEARS = np.arange(2018, 2023)
# CURRENT_BASELINE_YEARS = np.arange(2012, 2018)
def run_baseline(pars):
    df = pd.DataFrame()
    if "nitrogen_dap" in pars:
        nitro = list(zip(pars["nitrogen_dap"], pars["nitrogen_rate"]))
    else:
        nitro = [(0, pars["nitrogen"]),]
    for year in CURRENT_BASELINE_YEARS:
        tmp_df = run_spatial_dssat(
            dbname=dbname, 
            schema="kenya", 
            admin1=pars["admin1"],
            plantingdate=datetime(year, pars["plantingdate"].month, pars["plantingdate"].day),
            cultivar=cultivar_types[pars["cultivar"]],
            nitrogen=nitro,
            overview=False,
            all_random=False
        )
        tmp_df["year"] = year 
        df = pd.concat([df, tmp_df], ignore_index=True)
    return df

In [12]:
pars = {
    "plantingdate": datetime(2022, 4, 1), "cultivar": "Medium", "nitrogen": 40., 
    "admin1": "Nakuru", "nitrogen_dap": (5, 30, 50), "nitrogen_rate": (20, 10, 10)
}

In [13]:
baseline_df = run_baseline(pars)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 24.05it/s]


In [14]:
QUANTILES_TO_COMPARE = np.arange(0.025, 1, 0.05)
def baseline_quantile_stats(baseline_df):
    baseline_df["HARWT"] = baseline_df.HARWT.astype(int)
    baseline_stats = (
        baseline_df
        .groupby(["year"]).HARWT
        .quantile(QUANTILES_TO_COMPARE)
        .reset_index().rename(columns={"level_1": "quantile"})
        .groupby("quantile").HARWT
        .agg(["mean", "std"])
    )
    return baseline_stats
baseline_stats = baseline_quantile_stats(baseline_df)

In [19]:
def run_anomalies(df, baseline_stats):
    df["HARWT"] = df.HARWT.astype(int)
    run_stats = (
        df.HARWT
        .quantile(QUANTILES_TO_COMPARE)
        .reset_index().rename(columns={"index": "quantile"})
        .set_index("quantile")
    )
    anomalies = (run_stats.HARWT - baseline_stats["mean"])/baseline_stats["std"]
    return anomalies
anomalies = run_anomalies(baseline_df, baseline_stats)

In [16]:
from highcharts_core.chart import Chart
from highcharts_core.chart import HighchartsOptions
from highcharts_core.options.series.bar import ColumnSeries
from highcharts_core.options.plot_options.bar import ColumnOptions

In [17]:
from matplotlib.cm import get_cmap
from matplotlib.colors import to_hex

colors = list(map(to_hex, get_cmap("tab10").colors))
len(colors)

10

In [34]:
CAT_NAMES = ["Very low", "Low", "Normal", "High", "Very high"]
CAT_COLORS = ['#cc0000', "#ff9933", "#ffff66", "#99cc00", "#009933"]

def init_anomalies_chart():
    my_chart = Chart()
    my_chart.options = HighchartsOptions()
    my_chart.options.title = {
        'text': 'DSSAT maize yield anomaly', 
        "style": {
            "font-size": "15px"
        }
    }
    my_chart.options.y_axis = {
        "title": {
            'text': 'Probability (%)', 
            "style": {
                "font-size": "15px"
            }
        },
        "labels": {
            "style": {
                "font-size": "15px",
            }
        },
        "max": 100
    }
    my_chart.options.x_axis = {
        "title": {
            'text': 'Experiment', 
            "style": {
                "font-size": "15px",
            }
        },
        "labels": {
            "style": {
                "font-size": "15px",
            }
        }
    }
    for name, color in zip(CAT_NAMES[::-1], CAT_COLORS[::-1]):
        box = ColumnSeries()
        box.name = name
        box.color = color
        box.data = []
        my_chart.add_series(box)
        box.stacking = 'normal'
        box.data_labels = {
                "enabled": True
            }
    
    return my_chart

In [21]:
DEV_STAGES = [
    'Emergence-End Juvenile', 'End Juvenil-Floral Init',
    'Floral Init-End Lf Grow', 'End Lf Grth-Beg Grn Fil',
    'Grain Filling Phase'
]
DEV_STAGES_LABELS = [
    "Emerg.-End<br>Juv.", "End Juv-<br>Flor Init", "Flor Init-<br>End Lf Gro",
    "End lf Gro-<br>Beg Grain<br>Fil", "Grain<br>Fill"
]
def init_stress_chart(stress_type):
    my_chart = Chart()
    my_chart.options = HighchartsOptions()
    my_chart.options.title = {
        'text': f'{stress_type} stress', 
        "style": {
            "font-size": "15px"
        }
    }
    my_chart.options.y_axis = {
        "title": {
            'text': f'Stress (%)', 
            "style": {
                "font-size": "15px"
            }
        },
        "labels": {
            "style": {
                "font-size": "15px",
            }
        },
        "max": 100
    }
    my_chart.options.x_axis = {
        "title": {
            'text': 'Crop Dev. Stage', 
            "style": {
                "font-size": "15px",
            }
        },
        "labels": {
            "style": {
                "font-size": "15px",
            },
            "auto_rotation_limit": 0,
            "allow_overlap": True
        },
        "categories": DEV_STAGES_LABELS
    }      
    return my_chart

In [32]:
import numpy as np
Z_LIM = 0.44 # Limit for what is considered "Normal". 0.44 Splits equal groups (terciles)
Z_EXT_LIM = 2 # Limit for extreme values
def assign_categories(data):
    data = np.array(data)
    very_low = (data < -Z_EXT_LIM).mean()*100
    low = (data < -Z_LIM).mean()*100 - very_low
    very_high = (data > Z_EXT_LIM).mean()*100
    high = (data > Z_LIM).mean()*100 - very_high
    norm = 100 - low - high - very_high - very_low
    return list(map(int, [very_high, high, norm, low, very_low]))


def add_bar(chart, data):
    cat_data = assign_categories(data)
    label = f"{pars['cultivar']}<br>{pars['plantingdate'].strftime('%b %d')}<br>{pars['nitrogen']}"
    if chart.options.series is None:
        x = 0
    else:
        x = len(chart.options.series)/2
    # Boxplot
    series = chart.to_dict()["userOptions"]["series"]
    for n, cat in enumerate(cat_data):
        series_data = series[n].get("data", [])
        series_data += [cat]
        series[n]["data"] = series_data
    chart.update_series(*series)
    

In [23]:
def process_overview(overview):
    overview = parse_overview("".join(overview))
    overview = overview.set_index(["RUN", 'devPhase']).astype(float).reset_index()
    overview["watStress"] = overview[['stressWatPho', 'stressWatGro']].max(axis=1)
    overview["nitroStress"] = overview[['stressNitPhto', 'stressNitGro']].max(axis=1)
    return overview

In [24]:
def add_stress_bar(chart, data):
    "data is pandas.Series with the dev stage as index"
    data = 100*data
    
    box = ColumnSeries()
    box.data = [data.to_dict().get(dev_st) for dev_st in DEV_STAGES]
    if chart.options.series is None:
        n = 0
    else:
        n = len(chart.options.series)
    box.name = f"Exp {n}"
    chart.add_series(box)

In [59]:
# chart = init_stress_chart("Water")
# add_stress_bar(chart, tmp_df.groupby("devPhase").watStress.mean())
# chart.display()

In [26]:
# chart = init_anomalies_chart()
# add_bar(chart, anomalies)
# chart.display()

In [27]:
import plotly.graph_objects as go
from IPython.display import display, clear_output

### Baseline for Nakuru:
40 kg/ha Nitrogen (https://api.hub.ifdc.org/server/api/core/bitstreams/cb2720dc-d0f2-444a-9472-7dbd9e3763e9/content)

Planted in April (https://ipad.fas.usda.gov/rssiws/al/crop_calendar/eafrica.aspx)

In [65]:
nakuru_12_20_yield = [3.33, 2.55, 1.84, 1.57, 2.36, 2.33, 3.75, 1.42]
uasin_gishu_12_18 = [2.82, 4.09, 3.93, 3.89, 3.95, 3.12, 4.26]

In [29]:
def reset_anom_chart(chart):
    series = chart.to_dict()["userOptions"]["series"]
    for n, serie in enumerate(series):
        series[n]["data"] = []
    chart.update_series(*series)
    
def reset_stress_chart(chart):
#     series = chart.to_dict()["userOptions"]["series"]
#     chart.update_series(*[])
    chart.options.series = None

In [77]:
pars = {
    "plantingdate": datetime(2022, 4, 1), "cultivar": "Medium", "nitrogen": 40., 
    "admin1": "Nakuru", "nitrogen_dap": (5, 30, 50), "nitrogen_rate": (20, 10, 10)
}
other_pars = {"baseline stats": baseline_stats, "latest run": None, "latest overview": None}
other_pars["pars_df"] = pd.DataFrame(columns=["Cultivar", "Planting", "Nitro dap", "Nitro rate"])

def on_value_change(change, par):
    pars[par] = change.new

region_picker = widgets.Dropdown(
    options=admin1_list,
    value=pars["admin1"],
    description='Admin1:',
    disabled=False,
)
region_picker.observe(lambda x: on_value_change(x, "admin1"), names='value')

plantingdate_picker = widgets.DatePicker(
    description='Planting date:',
    disabled=False,
    value=pars["plantingdate"]
)
plantingdate_picker.observe(lambda x: on_value_change(x, "plantingdate"), names='value')

cultivar_picker = widgets.Dropdown(
    options=['Short', 'Medium', 'Long'],
    description='Cultivar type:',
    disabled=False,
    value=pars["cultivar"]
)
cultivar_picker.observe(lambda x: on_value_change(x, "cultivar"), names='value')

nitrogen_slider = widgets.FloatSlider(
    min=0,
    max=120.0,
    step=0.1,
    description='Nitrogen rate:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    value=pars["nitrogen"]
)
nitrogen_slider.observe(lambda x: on_value_change(x, "nitrogen"), names='value')

run_button = widgets.Button(
    description='Run DSSAT',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run DSSAT',
)
clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear',
)

baseline_button = widgets.Button(
    description='Set Baseline',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Set Baseline',
)

pars_desc = widgets.HTML(
    value="<b>Baseline</b><br><b>Planting:</b> {0}, <b>Cultivar:</b> {1},<br><b>Nitrogen:</b> {2}, <b>County:</b> {3}".format(
        pars["plantingdate"], pars["cultivar"], list(zip(pars["nitrogen_dap"], pars["nitrogen_rate"])), pars["admin1"]
    )
)

nitrogen_dap = widgets.IntsInput(
    value=pars["nitrogen_dap"],
    min=0,
    max=120,
    format='d',
    description='N date(dap):'
)
nitrogen_dap.observe(lambda x: on_value_change(x, "nitrogen_dap"), names='value')

nitrogen_rate = widgets.IntsInput(
    value=pars["nitrogen_rate"],
    min=0,
    max=999,
    format='d',
    description='N rate:'
)
nitrogen_rate.observe(lambda x: on_value_change(x, "nitrogen_rate"), names='value')

irrigation = widgets.Checkbox(
    value=False,
    description='Irrigation',
    disabled=True,
    indent=False
)

output_yield = widgets.Output()
output_stress1 = widgets.Output()
output_stress2 = widgets.Output()

pars_table = widgets.HTML(value="")

select_box = widgets.VBox([
    widgets.HBox([
        widgets.VBox([
            widgets.HBox([run_button, clear_button]),
            baseline_button,
        ]),
        widgets.VBox([
            widgets.HBox([region_picker, plantingdate_picker]),
            widgets.HBox([cultivar_picker, irrigation]),
        ]),
    ]),
    widgets.HBox([
        pars_desc,
        widgets.VBox([
            widgets.HTML(value="<b>Fertilizer options</b>"),
            widgets.HBox([widgets.HTML(value="Timing (dap):"), nitrogen_dap]),
            widgets.HBox([widgets.HTML(value="Rate (kg N/ha):"), nitrogen_rate]),
        ])
    ]),
    
    widgets.HBox([
        output_yield,
        pars_table
    ]),
    widgets.HBox([
        widgets.HBox([output_stress1,]),
        widgets.HBox([output_stress2,])
    ])
])

chart = init_anomalies_chart()
stress_chart_1 = init_stress_chart("Water")
stress_chart_2 = init_stress_chart("Nitrogen")

def clear_figure(b):
    reset_anom_chart(chart)
    reset_stress_chart(stress_chart_1)
    reset_stress_chart(stress_chart_2)
    with output_yield:
        chart.display()
    with output_stress1:
        stress_chart_1.display()
    with output_stress2:
        stress_chart_2.display()
    other_pars["pars_df"] = pd.DataFrame(columns=["Cultivar", "Planting", "Nitro dap", "Nitro rate"])
    pars_table.value = ""

def on_clic_run(b):
    df, overview = run(pars)
    other_pars["latest run"] = df
    overview_df = process_overview(overview)
    other_pars["latest overview"] = overview
    anomalies = run_anomalies(df, other_pars["baseline stats"])
    with output_yield:
        add_bar(chart, anomalies)
        clear_output()
        chart.display()
    with output_stress1:
        add_stress_bar(stress_chart_1, overview_df.groupby("devPhase").watStress.mean())
        clear_output()
        stress_chart_1.display()
    with output_stress2:
        add_stress_bar(stress_chart_2, overview_df.groupby("devPhase").nitroStress.mean())
        clear_output()
        stress_chart_2.display()
    n_exp = len(stress_chart_1.options.series)
    other_pars["pars_df"].loc[n_exp-1] = (pars["cultivar"], pars["plantingdate"], pars["nitrogen_dap"], pars["nitrogen_rate"])
    pars_table.value = other_pars["pars_df"].to_html()
        
        
def on_clic_baseline(b):
    pars_desc.value ="<b>Baseline</b><br><b>Planting:</b> {0}, <b>Cultivar:</b> {1},<br><b>Nitrogen:</b> {2}, <b>County:</b> {3}".format(
        pars["plantingdate"], pars["cultivar"], list(zip(pars["nitrogen_dap"], pars["nitrogen_rate"])), pars["admin1"]
    )
    baseline_df = run_baseline(pars)
    other_pars["baseline stats"] = baseline_quantile_stats(baseline_df)
    
clear_button.on_click(clear_figure)
run_button.on_click(on_clic_run)
baseline_button.on_click(on_clic_baseline)
display(select_box, output_yield, output_stress1, output_stress2)

Output()

Output()

Output()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 24.55it/s]


In [75]:
other_pars["latest run"]

,RUN,CR,TRT,FLO,MAT,TOPWT,HARWT,RAIN,TIRR,CET,PESW,TNUP,TNLF,TSON,TSOC
0,1,MZ,1,87,-99,7051,2406,810,0,472,277,72,24,19441,196
1,2,MZ,2,54,124,4778,2740,138,0,231,145,51,30,14851,151
2,3,MZ,3,49,108,3849,2013,140,0,203,167,42,34,13831,141
3,4,MZ,4,49,105,3634,1834,186,0,231,186,48,22,12606,128
4,5,MZ,5,53,123,4339,2435,137,0,223,151,47,27,13403,135
5,6,MZ,6,49,109,5802,2774,505,0,300,204,66,20,13868,141
6,7,MZ,7,45,95,2800,1251,131,0,191,176,36,32,11697,118
7,8,MZ,8,65,161,7807,4703,454,0,445,199,79,25,15557,157
8,9,MZ,9,87,-99,7050,2399,810,0,472,278,72,24,19394,195
9,10,MZ,10,63,155,8156,4712,544,0,404,234,76,27,15892,160


In [74]:
print("".join(other_pars["latest overview"]))

*SIMULATION OVERVIEW FILE

*DSSAT Cropping System Model Ver. 4.8.2.000 -release  APR 23, 2024 21:26:51

*RUN   1        : SAMPLE 1                  MZCER048 EXPEFILE    1
 MODEL          : MZCER048 - Maize
 EXPERIMENT     : EXPEFILE MZ SPATIAL ANALYSES TEST CASE; FLORENCE, SOUTH CAROLI
 DATA PATH      : /tmp/dssatrun5TWOHZMG/
 TREATMENT  1   : SAMPLE 1                  MZCER048


 CROP           : Maize            CULTIVAR : MEDIUM SEASON    ECOTYPE :IB0001
 STARTING DATE  : FEB 23 2022
 PLANTING DATE  : FEB 23 2022      PLANTS/m2 :     4.0    ROW SPACING :  90.cm
 WEATHER        : SEAA   2022
 SOIL           : IB00000001     TEXTURE : Loam  -    ISRIC soilgrids + HC27
 SOIL INIT COND : DEPTH:200cm EXTR. H2O:250.4mm  NO3:  0.0kg/ha  NH4:  0.0kg/ha
 WATER BALANCE  : RAINFED
 IRRIGATION     : NOT IRRIGATED
 NITROGEN BAL.  : SOIL-N & N-UPTAKE SIMULATION; NO N-FIXATION
 N-FERTILIZER   :       40 kg/ha IN     3 APPLICATIONS
 RESIDUE/MANURE : INITIAL :     0 kg/ha ;       0 kg/ha IN     0 AP